In [1]:
import os 
import re
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [2]:
# Modified following https://www.youtube.com/watch?v=02cdCd43Ccc 
API_KEY = "NA" #Causes ChatOpenAI to look for local models. base-url is needed to use ollama
Model = 'llama3'
gpt_llm = ChatOpenAI(api_key = API_KEY,model= Model, base_url="http://localhost:11434/v1")
gpt_llm.invoke('what is a bot')

AIMessage(content='A bot, short for "robot," is a computer program that automates tasks or interacts with users in a predefined manner. Bots are often used to perform repetitive tasks, provide customer support, and engage in conversation with humans.\n\nBots can be categorized into different types based on their functionality:\n\n1. **Task-oriented bots**: These bots automate specific tasks, such as:\n\t* Sending notifications\n\t* Processing transactions\n\t* Filling out forms\n2. **Conversational bots**: These bots simulate human-like conversations with users, often through:\n\t* Chatbots (e.g., chat windows or voice assistants like Siri or Alexa)\n\t* Messaging platforms (e.g., Facebook Messenger or WhatsApp)\n3. **Agent bots**: These bots act as virtual customer service representatives, providing assistance and answers to user inquiries.\n4. **Data-driven bots**: These bots analyze data and make decisions based on that information.\n\nBots can be created using various programming l

In [ ]:
parser = StrOutputParser()
gpt_chain = gpt_llm|parser
gpt_chain.invoke('what is a shark')

In [ ]:
loader = TextLoader('data.txt',encoding = 'utf-8')
document = loader.load()

In [ ]:
spliter = RecursiveCharacterTextSplitter(chunk_size = 200,chunk_overlap = 50)
chunks = spliter.split_documents(document)
chunks[3]


In [ ]:
# vector_storage = FAISS.from_documents(chunks,OpenAIEmbeddings())
# retriever = vector_storage.as_retriever()

from langchain_community.embeddings import OllamaEmbeddings

ollama_emb = OllamaEmbeddings(base_url = 'http://localhost:11434', model="llama3")
# May need: 
# !pip install faiss-cpu
# or 
# !pip install faiss-gpu

vector_storage = FAISS.from_documents(chunks, ollama_emb)
retriever = vector_storage.as_retriever()

In [ ]:
retriever.invoke('what is the main characteristic of the satellite')

In [ ]:
template = ("""
You are AI-powered chatbot designed to provide 
information and assistance for customers
based on the context provided to you only. 
            
Context:{context}
Question:{question}
""")

In [ ]:
prompt = PromptTemplate.from_template(template=template)
prompt.format(
    context = ' Here is a context to use',
    question = ' This is a question to answer'
)

In [ ]:
result = RunnableParallel(context = retriever,question = RunnablePassthrough())
chain = result |prompt | gpt_llm | parser

In [ ]:
chain.invoke('What is the recommended material for satelites? ')

In [ ]:
chain.invoke('are there any precedents to wood satellites? Do they work?')

In [ ]:
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/url/
# !pip install unstructured
from langchain_community.document_loaders import UnstructuredURLLoader

loaderhmlt = UnstructuredURLLoader(urls = ["https://www.nature.com/articles/d41586-024-01456-z"])
data = loaderhmlt.load()
spliterhtml = RecursiveCharacterTextSplitter(chunk_size = 200,chunk_overlap = 50)
chunkhtml = spliterhtml.split_documents(data)
chunkhtml[3]

In [ ]:
# The article of interest is inclluded in the first 44 chunks 
vector_storagehtml = FAISS.from_documents(chunkhtml[:44], ollama_emb)
retrieverhtml = vector_storagehtml.as_retriever()

In [ ]:
resulthtml = RunnableParallel(context = retrieverhtml,question = RunnablePassthrough())
chainhtml = resulthtml |prompt | gpt_llm | parser
chain_htmlr = chainhtml.invoke('are there any precedents to wood satellites? Do they work?')
chain_htmlr

In [ ]:
chain_htmlr2 = chainhtml.invoke('What is the recommended material for satelites? ')
chain_htmlr2

In [ ]:
chain_htmlr2 = chainhtml.invoke('What is the recommended material for satelites? ')
chain_htmlr2

In [ ]:
chain_htmlr2 = re.sub(r'\. ',r'.\n', chain_htmlr2)
print(chain_htmlr2)

In [ ]:
chain_htmlr3 = chainhtml.invoke('What is the main idea of the context provided? ')
chain_htmlr3
chain_htmlr3 = re.sub(r'\. ',r'.\n', chain_htmlr3)
print(chain_htmlr3)